In [12]:
import spacy

In [20]:
import io
import os
from os import path

In [13]:
nlp = spacy.load('en_core_web_md')

In [17]:
# INPUT_DATA = "../data/raw_html_asda_yoghurt.html"       # .gz archive or directory of archives
INPUT_DATA_DIR = "../data/"
INPUT_DATA_FN = "SR-Leg_ASC/FOOD_DES"
INPUT_DATA = f"{INPUT_DATA_DIR}{INPUT_DATA_FN}.txt"
OUTPUT_FILE_DIR = "../assets/"  # path to output JSONL
# OUTPUT_FILE_FN = "raw_html_asda_yoghurt"  # path to output JSONL
OUTPUT_FILE = f"{OUTPUT_FILE_DIR}{INPUT_DATA_FN}.jsonl"  # path to output JSONL

In [1]:
import unicodedata

In [2]:
# function to remove accented characters
def remove_accented_chars(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


In [5]:
# imports
import re

In [9]:
# function to remove special characters
def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat, '', text)
def remove_nonutf8_characters(text):
    # define the pattern to keep
    utf8Pat = r'^(?!([\x00-\x7F]|([\xC2-\xDF]|\xE0[\xA0-\xBF]|\xED[\x80-\x9F]|(|[\xE1-\xEC]|[\xEE-\xEF]|\xF0[\x90-\xBF]|\xF4[\x80-\x8F]|[\xF1-\xF3][\x80-\xBF])[\x80-\xBF])[\x80-\xBF]))*$'
    return re.sub(utf8Pat, '', text)
 
# call function
remove_nonutf8_characters("Please remove the nonutf8: Ä°nanÃ§ EsaslarÄ±Done? $500USD!")

'Please remove the nonutf8: Ä°nanÃ§ EsaslarÄ±Done? $500USD!'

In [3]:
# call function
remove_accented_chars('Sómě Áccěntěd těxt. Some words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.')

'Some Accented text. Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.'

In [ ]:
home_dir = os.system("cd ~")
print("`cd ~` ran with exit code %d" % home_dir)
unknown_dir = os.system("cd doesnotexist")
print("`cd doesnotexis` ran with exit code %d" % unknown_dir)

In [28]:
def appendToFileName(fileName, appendstr):
    return appendstr.join(path.splitext(fileName))

In [18]:
import subprocess

The exit code was: 0total 200
-rw-r--r--  1 joeyd  staff   5008 13 Dec 13:12 01_Preprocess_Reddit.ipynb
-rw-r--r--  1 joeyd  staff   3147 11 Dec 12:03 02_Process_text_and_counts.ipynb
-rw-r--r--  1 joeyd  staff  75656 11 Dec 12:03 03_Generate_counts.ipynb
-rw-r--r--  1 joeyd  staff  10969 14 Dec 09:56 supermarket_product_ner.ipynb



In [34]:

_ = subprocess.run(' '.join([f"touch", appendToFileName(INPUT_DATA, '_clean'), 
                    "&&", 
                    "iconv", "-f utf-8", "-t utf-8", f"-c {INPUT_DATA} > {appendToFileName(INPUT_DATA, '_clean')}"]), shell=True)

In [37]:
INPUT_DATA = appendToFileName(INPUT_DATA, '_clean')

In [91]:
from bs4 import BeautifulSoup, SoupStrainer

In [99]:
only_header_tags = SoupStrainer(re.compile('h[1234]'))

with open("../data/raw_html_asda_yoghurt.html", 'rb') as f:
    soup_only_headers = BeautifulSoup(f.read(), 'html.parser', parse_only=only_header_tags)
    f.seek(0)
    soup = BeautifulSoup(f.read(), 'html.parser')


print(soup_only_headers)


<h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/910001440072?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Dairylea
														Cheese Triangles x16</a></h3><h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/1000210453264?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Bonne
														Maman Strawberry Mousse with Strawberry Pieces</a></h3><h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/910000329132?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Cathedral
														City Extra Mature Cheddar Cheese</a></h3><h3 class="co-product__title pdp-personal-recs__prod-title"><a class=

In [73]:
def minTabsAndLines(pretty_soup_dont_process):
    pretty_soup_dont_process = re.sub(r'\n\t+', ' ', pretty_soup_dont_process)
    # pretty_soup_dont_process = re.sub(r'\t', '-', pretty_soup_dont_process)
    # pretty_soup_dont_process = re.sub(r'\n', '*', pretty_soup_dont_process)
    pretty_soup_dont_process = re.sub(r'\t+', '\t', pretty_soup_dont_process)
    pretty_soup_dont_process = re.sub(r'\n+', '\n', pretty_soup_dont_process)
    return pretty_soup_dont_process
# print(minTabsAndLines(soup.get_text()))


In [113]:
# import unicode
# with open(INPUT_DATA, 'r', encoding='utf-8') as f:
#     doc_generator = nlp.pipe(f.readlines())
doc_generator = nlp(minTabsAndLines(soup.get_text()))
# doc_only_headers = nlp(minTabsAndLines(str(soup_only_headers)))
doc_only_headers_generator = nlp.pipe((tag.string for tag in soup_only_headers.find_all(re.compile(r'h[1234]'))))
title_doc = nlp(soup.title.get_text())
# print(str(soup_only_headers))

In [78]:
if title_doc.ents:
    spacy.displacy.render(title_doc, style="ent")
else:
    print('No Named Entities recognised in title.')

In [127]:
# print(soup.find_all(re.compile('h[1234]')))
for doc in doc_only_headers_generator:
    print(str(doc))
list([tag.string for tag in soup_only_headers.find_all(re.compile(r'h[1234]'))][0].parents)

[<a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/910001440072?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Dairylea
 														Cheese Triangles x16</a>,
 <h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/910001440072?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Dairylea
 														Cheese Triangles x16</a></h3>,
 <h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/910001440072?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357">Dairylea
 														Cheese Triangles x16</a></h3><h3 class="co-product__title pdp-personal-recs__prod-title"><a class="co-product__anchor" data-auto-id="linkProductTitle" href="/product/1000210453264?origin=/pr

In [85]:
'''If Product text is linked, then probably not the lead product on the page, we should look for unlinked header text'''
for link in soup.find_all('a'):
    print(minTabsAndLines(link.get_text().strip()), ' : ', link.get('href'))

Skip to Main Content  :  #main-content
Dairylea Cheese Triangles x16  :  /product/910001440072?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357
Bonne Maman Strawberry Mousse with Strawberry Pieces  :  /product/1000210453264?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357
Cathedral City Extra Mature Cheddar Cheese  :  /product/910000329132?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357
Activia Peach No Added Sugar Gut Health Yogurt  :  /product/1000290067858?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357
Babybel Mini Original x12  :  /product/465888?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357
Carnival Kitchen Jerk Chicken Flatbread  :  /product/1000317681229?origin=/product/natural-plain-organic/fage-total-fat-free-greek-recipe-natural-yogurt/24771357

In [112]:
# print(doc_generator.ents)
# print('\n-----------------------------------------------------------\n')
# spacy.displacy.render(doc_generator, style="ent")
spacy.displacy.render(doc_only_headers, style="ent")

In [40]:
i = 0
for doc in doc_generator:
    if doc.ents:
        print(doc.ents)
    i += 1
    if i > 500 and doc.ents:
        break

(8.79, 3.87)
(WHIPPED, W/ SALT~^^^~Y~^^0^^6.38^^^)
(4.27, 8.79, 3.87)
(blue~^~CHEESE,BLUE~^^^~Y~^^0^^6.38^, 4.27, 8.79, 3.87)
(~01005~^~0100~^~Cheese, brick~^~CHEESE, BRICK~^^^~Y~^^0^^6.38, 4.27, 8.79, 3.87)
(~01006~^~0100~^~Cheese, brie~^~CHEESE,BRIE~^^^~Y~^^0^^6.38^, 4.27, 8.79, 3.87)
(4.27, 8.79, 3.87)
(caraway~^~CHEESE, 4.27, 8.79, 3.87)
(USDA, Food Distribution Program)~^~CHEESE)
(4.27, 8.79, 3.87)
(COLBY~^^^~Y~^^0^^6.38^, 4.27, 8.79, 3.87)
(COTTAGE, CRMD, LRG, SML, 8.79, 3.87)
(COTTAGE, CRMD, W/FRUIT~^^^~Y~^^0^^6.38^, 8.79, 3.87)
(COTTAGE, NONFAT, UNCRMD, DRY, LRG, SML, 8.79, 3.87)
(2%, COTTAGE, LOWFAT,2%)
(~01016~^~0100~^~Cheese, 1%, COTTAGE, LOWFAT,1%, 4.27, 8.79, 3.87)
(4.27, 8.79, 3.87)
(edam~^~CHEESE, 4.27, 8.79, 3.87)
(~01019~^~0100~^~Cheese, feta~^~CHEESE, FETA~^^^~Y~^^0^^6.38^4.27^, 8.79, 3.87)
(fontina~^~CHEESE, FONTINA~^^^~Y~^^0^^6.38^, 4.27, 8.79, 3.87)
(4.27, 8.79, 3.87)
(~01022~^~0100~^~Cheese, gouda~^~CHEESE, 4.27, 8.79, 3.87)
(~01023~^~0100~^~Cheese, gruyere~^~CHEE